# 壁伸ばし法(迷路生成アルゴリズム)

ここで生成される迷路とは、2 次元配列に 0 と 1 で構成されます。1 が通れない壁部分で 0 が通れる通路部分を意味します。

```text
1111111
1000001
1010111
1010001
1011101
1010001
1111111
```

迷路の成立条件として、以下の二つがある

1. 外の通路につながらない死に領域(閉じた領域)がない
2. 通路がループにならない

# 壁伸ばし法の手順

1. 迷路全体を構成する 2 次元配列を、幅高さ 5 以上の奇数で生成します。
2. 迷路の外周を壁とし、それ以外を通路とします。
3. x, y ともに偶数となる座標を壁伸ばし開始座標(候補)としてリストアップします。
4. 壁伸ばし開始座標からランダムで座標を取り出し、通路の場合のみ 5. の壁伸ばし処理を行います。すべての候補座標が壁になるまで繰り返します。
5. 壁伸ばし処理を行います。
   - 指定座標を壁とします。
   - 次に掘り進める方向(隣のセルが通路の方向かつ 2 セル先が現在拡張中の壁ではない方向)をランダムで決定しします。
   - 拡張する方向 2 セル先が壁の場合(既存の壁に接続された場合)、壁の拡張を終了します。
   - 通路の場合、そのセルから続けて拡張します。(5. の処理を再帰的に呼び出す。)
   - 四方がすべて現在拡張中の壁の場合、拡張できる座標が見つかるまで、現在拡張中の壁をバックして、壁の拡張を再開します。
   - すべての候補座標が壁(拡張済)になれば完了です。


In [24]:
import random

# 自分で実装したもの(未完成)


def maze_generator_extend_original(size):
    path = "_"
    wall = "■"

    # サイズが5未満では迷路を生成できない(壁と繋がってしまうため)
    if size < 5:
        size = 5
    if size % 2 == 0:
        size += 1

    maze = [[""] * (size) for _ in range(size)]
    # 壁伸ばしの開始地点候補
    candidates = {}

    # 迷路の枠の作成
    for x in range(size):
        for y in range(size):
            if x == 0 or y == 0 or x == size - 1 or y == size - 1:
                maze[x][y] = wall
            else:
                maze[x][y] = path

                # 開始位置の候補
                if x % 2 == 0 and y % 2 == 0:
                    point = (x, y)
                    candidates[point] = False

    def get_random_next_point(x, y):
        next_x = x
        next_y = y
        next_next_x = x
        next_next_y = y

        d = random.randint(0, 3)
        if d == 0:
            next_x += 1
            next_next_x += 1
        elif d == 1:
            next_x -= 1
            next_next_x -= 1
        elif d == 2:
            next_y += 1
            next_next_y += 1
        elif d == 3:
            next_y -= 1
            next_next_y -= 1

        return next_x, next_y, next_next_x, next_next_y

    def generate_wall(x, y):
        # スタート地点を壁に変更
        maze[x][y] = wall

        # 現在伸ばしている途中の壁を管理する
        extending_walls = {}

        while True:
            # 伸ばす方向を決定する
            next_x, next_y, next_next_x, next_next_y = get_random_next_point(
                x, y)

            if extending_walls.get((next_x, next_y), False) == True:
                continue

            if maze[next_next_x][next_next_y] == wall:
                break

            extending_walls[(next_x, next_y)] = True
            x, y = next_x, next_y

        # 伸ばした壁をmazeに反映する
        for point in extending_walls.keys():
            maze[point[0]][point[1]] = wall

    # # 迷路の作成
    for x in range(2, size - 1, 2):
        for y in range(2, size - 1, 2):
            if candidates[(x, y)] == False:
                generate_wall(x, y)
    # generate_wall(2,2)

    for x in maze:
        print(*x)


maze_generator_extend_original(15)

■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■
■ _ ■ _ _ _ ■ ■ ■ _ ■ _ _ _ ■
■ _ ■ ■ ■ _ ■ ■ ■ _ ■ ■ ■ _ ■
■ _ ■ ■ ■ _ ■ _ ■ ■ ■ ■ _ _ ■
■ _ ■ _ ■ ■ ■ ■ ■ ■ ■ ■ ■ _ ■
■ _ _ _ ■ ■ _ ■ ■ ■ ■ ■ ■ _ ■
■ _ ■ _ ■ ■ ■ _ ■ _ ■ ■ ■ _ ■
■ ■ ■ _ _ _ ■ _ _ _ ■ ■ _ _ ■
■ ■ ■ _ ■ ■ ■ _ ■ _ ■ _ ■ ■ ■
■ ■ ■ ■ ■ ■ ■ _ ■ ■ ■ _ ■ ■ ■
■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ _ ■ _ ■
■ _ _ _ ■ ■ ■ ■ ■ _ _ ■ ■ ■ ■
■ ■ ■ _ ■ ■ ■ ■ ■ _ ■ ■ ■ ■ ■
■ ■ _ _ _ _ _ _ _ _ _ ■ ■ _ ■
■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■


In [34]:
import random
from enum import Enum
from typing import List
from collections import deque


class Direction(Enum):
    Up = 0,
    Right = 1,
    Down = 2,
    Left = 3


class Cell:

    def __init__(self, x: int, y: int) -> None:
        self.X = x
        self.Y = y


# サイトを見て実装したもの
def maze_generator_extend(size):
    path = "_"
    wall = "■"

    # サイズが5未満では迷路を生成できない(壁と繋がってしまうため)
    if size < 5:
        size = 5
    if size % 2 == 0:
        size += 1

    maze = [[""] * (size) for _ in range(size)]
    # 壁伸ばしの開始地点候補
    startCells: List[Cell] = []
    # 現在拡張中の壁の情報を保持
    currentWallCells = deque()

    def setWall(x: int, y: int):
        maze[x][y] = wall
        if x % 2 == 0 and y % 2 == 0:
            currentWallCells.append(Cell(x, y))

    def extend_wall(x: int, y: int):
        directions = []
        if maze[x][y - 1] == path and Cell(x, y - 2) not in currentWallCells:
            directions.append(Direction.Up)
        if maze[x + 1][y] == path and Cell(x + 2, y) not in currentWallCells:
            directions.append(Direction.Right)
        if maze[x][y + 1] == path and Cell(x, y + 2) not in currentWallCells:
            directions.append(Direction.Down)
        if maze[x - 1][y] == path and Cell(x - 2, y) not in currentWallCells:
            directions.append(Direction.Left)

        # ランダムにマスを伸ばす
        if len(directions) > 0:
            setWall(x, y)

            # 伸ばす先が通路の場合は拡張を続ける
            is_path = False
            direction_index = random.randint(0, len(directions) - 1)

            if directions[direction_index] == Direction.Up:
                is_path = maze[x][y - 2] == path
                y -= 1
                setWall(x, y)
                y -= 1
                setWall(x, y)
            elif directions[direction_index] == Direction.Right:
                is_path = maze[x + 2][y] == path
                x += 1
                setWall(x, y)
                x += 1
                setWall(x, y)
            elif directions[direction_index] == Direction.Down:
                is_path = maze[x][y + 2] == path
                y += 1
                setWall(x, y)
                y += 1
                setWall(x, y)
            elif directions[direction_index] == Direction.Left:
                is_path = maze[x - 2][y] == path
                x -= 1
                setWall(x, y)
                x -= 1
                setWall(x, y)

            if is_path:
                extend_wall(x, y)

        else:
            before_cell = currentWallCells.pop()
            extend_wall(before_cell[0], before_cell[1])

    # 迷路の枠の作成
    for x in range(size):
        for y in range(size):
            if x == 0 or y == 0 or x == size - 1 or y == size - 1:
                maze[x][y] = wall
            else:
                maze[x][y] = path

                # 開始位置の候補
                if x % 2 == 0 and y % 2 == 0:
                    cell = Cell(x, y)
                    startCells.append(cell)

    # 壁が拡張できなくなるまでループ
    while len(startCells) > 0:
        # 開始位置をランダムに取得
        start_index = random.randint(0, len(startCells) - 1)
        cell = startCells[start_index]
        del startCells[start_index]

        x = cell.X
        y = cell.Y

        if maze[x][y] == path:
            currentWallCells.clear()
            extend_wall(x, y)

    # 迷路の出力
    for x in maze:
        print(*x)


maze_generator_extend(40)

■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■
■ _ _ _ _ _ ■ _ _ _ _ _ _ _ _ _ _ _ _ _ ■ _ ■ _ _ _ ■ _ _ _ _ _ _ _ _ _ _ _ ■ _ ■
■ ■ ■ ■ ■ _ ■ _ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ _ ■ _ ■ _ ■ ■ ■ _ ■ ■ ■ ■ ■ ■ ■ _ ■ _ ■ _ ■
■ _ _ _ ■ _ _ _ ■ _ ■ _ ■ _ _ _ ■ _ ■ _ _ _ ■ _ _ _ ■ _ ■ _ _ _ _ _ ■ _ ■ _ _ _ ■
■ _ ■ ■ ■ ■ ■ _ ■ _ ■ ■ ■ _ ■ ■ ■ _ ■ ■ ■ _ ■ ■ ■ _ ■ _ ■ ■ ■ _ ■ ■ ■ ■ ■ _ ■ ■ ■
■ _ _ _ ■ _ _ _ _ _ _ _ _ _ _ _ _ _ ■ _ _ _ _ _ _ _ _ _ _ _ ■ _ _ _ ■ _ ■ _ _ _ ■
■ ■ ■ _ ■ ■ ■ ■ ■ _ ■ ■ ■ ■ ■ _ ■ ■ ■ _ ■ ■ ■ ■ ■ _ ■ _ ■ ■ ■ _ ■ _ ■ _ ■ _ ■ ■ ■
■ _ _ _ _ _ _ _ ■ _ _ _ ■ _ ■ _ _ _ ■ _ _ _ ■ _ _ _ ■ _ _ _ ■ _ ■ _ _ _ ■ _ ■ _ ■
■ _ ■ ■ ■ _ ■ ■ ■ _ ■ _ ■ ■ ■ _ ■ ■ ■ _ ■ ■ ■ ■ ■ _ ■ ■ ■ ■ ■ _ ■ ■ ■ _ ■ _ ■ _ ■
■ _ ■ _ ■ _ _ _ ■ _ ■ _ _ _ ■ _ ■ _ _ _ _ _ ■ _ _ _ ■ _ _ _ ■ _ ■ _ ■ _ _ _ _ _ ■
■ _ ■ _ ■ ■ ■ ■ ■ _ ■ _ ■ _ ■ _ ■ _ ■ _ ■ ■ ■ ■ ■ _ ■ ■ ■ ■ ■ _ ■ ■ ■ _ ■ ■ ■ ■ ■
■ _ _ _ ■ _ _ _ _ _ ■ _ ■ _ ■ _ ■ _ ■ _ ■ _ ■ _ ■ _ _ _ _ _ _ _ ■ _ _ _ _ _ ■ _ ■
■ _ ■ ■ ■ _ ■ ■ 